<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Госпрограммы" data-toc-modified-id="Госпрограммы-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Госпрограммы</a></span></li><li><span><a href="#Планы-реализации-госпрограмм" data-toc-modified-id="Планы-реализации-госпрограмм-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Планы реализации госпрограмм</a></span></li><li><span><a href="#Порядки-предоставления-субсидий" data-toc-modified-id="Порядки-предоставления-субсидий-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Порядки предоставления субсидий</a></span></li></ul></div>

In [1]:
import os
import shutil

import time
import datetime
from datetime import date,datetime

import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
'''Получение перечня файлов'''
def GetFileList(directory):
   
    #Получаем список файлов в переменную files
    files = os.listdir(directory)
    files
    #Получаем список файлов .xlsx
    reports = []
    for file in files:
        if file.endswith((".xlsx",".xlsm")):
            reports.append(file)
    return reports

In [3]:
def get_info_from_dir(directory, template_ind, drop_list, change_dict):
    template = pd.read_excel(f'{templates_dir}{templates[template_ind]}', sheet_name='ЧЕК-ЛИСТ2', header=4)
    main_col = 'Наименование требования'
    template = template[[main_col]]
    
    print(f'Количество чек-листов в папке: {len(GetFileList(directory))}\n')
    result = template.copy()

    for file_ind, file in enumerate(GetFileList(directory)[:]):
        try:
            temp = pd.read_excel(f'{directory}{file}', sheet_name='ЧЕК-ЛИСТ2', header=4)
            temp.rename(columns={'Статус': file}, inplace=True)
            main_col = 'Наименование требования'
            temp = temp[[main_col, file]]

            temp = temp[~temp[main_col].isin(drop_list)]        
            for key in change_dict:
                temp.loc[temp[temp[main_col].isin([key])].index, main_col] = change_dict[key]


            temp_col = temp[main_col].unique()
            result_col = result[main_col].unique()
            check = list(set(temp_col) - set(result_col))
            check_ = list(set(result_col) - set(temp_col))
#             if (check!=[]) or (check_!=[]):
#                 print(f'{file}\n')
#                 if check!=[]:
#                     print(f'Необходимо удалить/переименовать: {check}\n')
#                 if check_!=[]:
#                     print(f'Необходимо добавить: {check_}\n')            
#             else:
#                 result = pd.merge(result, temp, how='outer', on=main_col)                
            if (check!=[]):
                print(f'{file}\n')
                print(f'Необходимо удалить/переименовать: {check}\n')
            else:
                result = pd.merge(result, temp, how='outer', on=main_col)

        except PermissionError:
            print(f'\033[93m{file} редактируется другим пользователем\033[0m\n')
            
    result = result.T
    result.columns = result.iloc[0]
    result['file'] = result.index
    result.index = range(len(result.index))
    result.drop([0], inplace=True)
    result.index = range(len(result.index))
    if len(result) == len(GetFileList(directory)):
        print('Успешное завершение!')
        return result
    else:
        print('Загружены не все данные! Есть проблемы с названиями столбцов!')
        return None        

In [4]:
"""Шаблоны"""
templates_dir = r'J:\~Шаблоны/'
templates = GetFileList(templates_dir)
templates

['Шаблон заключения на госпрограмму.xlsx',
 'Шаблон заключения на план реализации.xlsx',
 'Шаблон заключения на порядок предоставления субсидий.xlsx']

In [5]:
"""Сохранение результатов"""
# Folder = r'U:\~ 09_Машинное обучение_Прогноз показателей СЭР\ЧЕК-ЛИСТЫ и DATA-SHOP\ВЫГРУЗКА ЧЕК-ЛИСТОВ/'
Folder = r'U:\ЧЕК-ЛИСТЫ и DATA-SHOP\ВЫГРУЗКА ЧЕК-ЛИСТОВ/'
File_name = 'RESULTS - Чек-листы.xlsx'

# Госпрограммы

In [6]:
template_ind = 0
directory = r'J:\~Шаблоны\Чек-листы\Госпрограммы/'

drop_list = []
change_dict = {
    '1.5. Ожидаемые результаты реализации госпрограммы/подпрограммы установлены на основе из плановых значений индикаторов (показателей)': 
    '1.5. Ожидаемые результаты реализации госпрограммы/подпрограммы установлены на основе плановых значений индикаторов (показателей)',
    '1.6.1. соответствуют лимитам, бюджету ': '1.6.1. соответствуют бюджетной росписи',      
}

In [7]:
result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 83

Успешное завершение!


In [8]:
with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
        result.to_excel(writer, sheet_name = 'ГП_' + str(date.today()), header=True, index=False, encoding = '1251')

# Планы реализации госпрограмм

In [9]:
template_ind = 1
directory = r'J:\~Шаблоны\Чек-листы\План реализации/'

drop_list = []
change_dict = {
    'Внесение изменений вплан реализации не предполагает внесение изменений в постановление по утчерждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)':
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утверждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)',
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утчерждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)':
    'Внесение изменений в план реализации не предполагает внесение изменений в постановление по утверждению государственной программы по основаниям, предусмотренным п. 16.1 Порядка (см. внизу таблицы)',
    
    
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:\nзаключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события: заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события: заключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий региональных проектов, предусматривающих строительно-монтажные работы, выделены средующие контрольные события:',
    
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:формирование земельного участка;проектирование;объявление конкурсов;заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ': 
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:\nформирование земельного участка; проектирование; объявление конкурсов; заключение государственного контракта; подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события: формирование земельного участка;проектирование;объявление конкурсов;заключение государственного контракта;подписание промежуточных или итоговых актов выполненных работ':
    'Для мероприятий, предусматривающих осуществление строительно-монтажных работ не в рамках региональных проектов, выделены следующие контрольные события:',
    
    'Контрольное событие предусмотрено в декабре:\nпри наличии контрольных событий в предыдущих периодах;для новых мероприятий, которые были включены при уточнении бюджета в осеннюю сессию;контрольное событие подразумевает проведение события (семенара, конференции, торжественного собрания и т.п.)':
    'Контрольное событие предусмотрено в декабре:',
    'Контрольное событие предусмотрено в декабре: при наличии контрольных событий в предыдущих периодах;для новых мероприятий, которые были включены при уточнении бюджета в осеннюю сессию;контрольное событие подразумевает проведение события (семенара, конференции, торжественного собрания и т.п.)':
    'Контрольное событие предусмотрено в декабре:',        
}

In [10]:
result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 88

Успешное завершение!


In [11]:
with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
        result.to_excel(writer, sheet_name = 'ПР_' + str(date.today()), header=True, index=False, encoding = '1251')

# Порядки предоставления субсидий

In [12]:
template_ind = 2
directory = r'J:\~Шаблоны\Чек-листы\Субсидии/'

drop_list = ['Номер и дата версии в АИС Проект-СМАРТ Про', 
             'Дата в АИС Проект-СМАРТ Про',
             'Плановый год госпрограммы', 
             # Общие требования
             'Регламентирован весь период с даты объявления приема заявлений от претендентов до даты перечисления субсидии (гранта) получателю.',
             'в т.ч. срок приема документов от претендентов на получение субсидии (гранта) составляет не менее 10 рабочих дней.', 
             'Укажите количество потенциальных получателей субсидии (гранта)',
             'Установлен размер субсидии и (или) порядок расчета размера субсидии с указанием информации, обосновывающей ее размер (формулы расчета и порядок ее применения и т.п.).',
             'Установлен размер субсидии и (или) порядок расчета размера субсидии с указанием информации, обосновывающей ее размер (формулы расчета и порядок ее применения и т.п.).',
             # Гранты в форме субсидий
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)\nСубсидии производителям товаров (работ, услуг) (постановление от 06.09.2016 г. №887)\nСубсидии некоммерческим организациям (постановление от 07.05.2017 г. №541)',
             'Гранты в форме субсидий (постановление 27.03.2019 г. № 322)\nСубсидии производителям товаров (работ, услуг) (постановление от 06.09.2016 г. №887)\nСубсидии некоммерческим организациям (постановление от 07.05.2017 г. №541)',
             'Определен порядок представления получателем субсидии/гранта отчетности о достижении результатов (показателей)', 
             'Определен срок представления получателем субсидии/гранта отчетности о достижении результатов (показателей), об осуществлении расходов',
             'Определена форма представления получателем субсидии/гранта отчетности о достижении результатов (показателей), об осуществлении расходов',             
             'Установлено право главного распорядителя устанавливать в соглашении сроки и формы представления получателем субсидии/гранта дополнительной отчетности',
             'Только для грантов в форме субсидий:',                                                 
            ]
change_dict = {'Наименование постановления': 'Наименование приказа',
               'Номер РКПД в СЭД Дело': 'Номер и дата РКПД в СЭД Дело',
               'Указаны результаты предоставления гранта/субсидии и при необходимости показатели, необходимые для достижения результатов предоставления гранта/субсидии.': 'Указаны результаты предоставления гранта/субсидии.',
               'Указаны результаты предоставления гранта и при необходимости показатели, необходимые для достижения результатов предоставления гранта.': 'Указаны результаты предоставления гранта/субсидии.',
               'Результаты и (или) показатели конкретны': 'Результаты конкретны',
               'Результаты и (или) показатели измеримы': 'Результаты измеримы',
               'Результаты и (или) показатели соответствуют федеральным, региональным проектам': 'Результаты и (или) показатели соответствуют федеральным, региональным проектам или государственным программам',               
              }

In [13]:
result = get_info_from_dir(directory, template_ind, drop_list, change_dict)

Количество чек-листов в папке: 46

Успешное завершение!


In [14]:
with pd.ExcelWriter(Folder+File_name, engine="openpyxl", mode='a') as writer:
        result.to_excel(writer, sheet_name = 'Субсидии_' + str(date.today()), header=True, index=False, encoding = '1251')